# Test detection model

In [3]:
%run '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/K2TranPix_test.py'
%run '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/Injections/KasenKilonovaSeed.ipynb'
%matplotlib notebook
from tqdm import tnrange, tqdm_notebook
from random import randint
save = '/Users/ryanr/Documents/PhD/coding/Kepler/pipelines/Injections/Kilonova/'
D = np.arange(1,120,1)*1e6 # distance in Mpc

pixfile = '/Volumes/TOSHIBA EXt/K2/c6/212200000/72000/ktwo212272599-c06_lpd-targ.fits.gz'
hdu = fits.open(pixfile)
dat = hdu[1].data
data = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data#np.copy(testdata)#
time = dat["TIME"] + 2454833.0
Qual = hdu[1].data.field('QUALITY')


funny_keywords = {'1CTYP4': 'CTYPE1',
                  '2CTYP4': 'CTYPE2',
                  '1CRPX4': 'CRPIX1',
                  '2CRPX4': 'CRPIX2',
                  '1CRVL4': 'CRVAL1',
                  '2CRVL4': 'CRVAL2',
                  '1CUNI4': 'CUNIT1',
                  '2CUNI4': 'CUNIT2',
                  '1CDLT4': 'CDELT1',
                  '2CDLT4': 'CDELT2',
                  '11PC4': 'PC1_1',
                  '12PC4': 'PC1_2',
                  '21PC4': 'PC2_1',
                  '22PC4': 'PC2_2'}
mywcs = {}
for oldkey, newkey in funny_keywords.items():
    mywcs[newkey] = hdu[1].header[oldkey]
wcs = WCS(mywcs)


Result = []
Fakes = []
for distance in tqdm_notebook(D):
    Run = []
    Run_fakes = []
    for i in tqdm_notebook(range(1000),leave = False):
        
        xposition = randint(0,data.shape[1]-1)
        yposition = randint(0,data.shape[2]-1)
        Position = [xposition,yposition]
        Start = randint(0,data.shape[0]-1)
        
        Kilonova, Kilotime = K2Kilonova(3,distance)
        Kilonova = Kilonova[0,Kilotime > 0]
        if Kilonova.shape[0] > data[Start:].shape[0]:
            Kilonova = Kilonova[:data[Start:].shape[0]]
        else:
            temp = np.zeros(data[Start:].shape[0])
            temp[:Kilonova.shape[0]] = Kilonova
            Kilonova = temp
        testdata = np.copy(data)
        testdata[Start:,Position[0],Position[1]] = testdata[Start:,Position[0],Position[1]] + Kilonova
        detec, falsedetec = K2tranPix(testdata,time,Qual,wcs,Position,Start)
        Run.append(detec)
        Run_fakes.append(falsedetec)
    Result.append(Run)
    Fakes.append(Run_fakes)
Result = np.array(Result)
Fakes = np.array(Fakes)
    

HBox(children=(IntProgress(value=0, max=119), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

KeyboardInterrupt: 

In [9]:
from mpi4py import MPI
import time
dims = 1 # set to be length of your task
start = time.time()

def print_mpi(string):
    comm = MPI.COMM_WORLD
    print("["+str(comm.Get_rank())+"] "+string)

def print_master(string):
    comm = MPI.COMM_WORLD
    if comm.Get_rank() == 0:
        print("["+str(comm.Get_rank())+"] "+string)

# init MPI

comm = MPI.COMM_WORLD
nPE = comm.Get_size()
myPE = comm.Get_rank()
print_master("Total number of MPI ranks = "+str(nPE))
comm.Barrier()

# Remove previous test files

if comm.Get_rank() == 0:
    os.system("mv *cpu.txt oldouts/")
    os.system("mv total_prog.txt oldouts/")

# Progress saving function

def writemyprog(filename,out):
    np.savez(filename,out)

# domain decomposition

my_start = myPE * (dims / nPE);
my_end   = (myPE+1) * (dims / nPE) - 1;
# last PE gets the rest
if (myPE == nPE-1): my_end = dims-1;
print_mpi("my_start = "+str(my_start)+", my_end = "+str(my_end))

# parallel loop

filename = "my_prog_"+str(myPE).zfill(4)


for n in range(my_start, my_end+1):
    mytimestart = time.time()
        
    xposition = randint(0,data.shape[1]-1)
    yposition = randint(0,data.shape[2]-1)
    Position = [xposition,yposition]
    Start = randint(0,data.shape[0]-1)

    Kilonova, Kilotime = K2Kilonova(3,distance)
    Kilonova = Kilonova[0,Kilotime > 0]
    if Kilonova.shape[0] > data[Start:].shape[0]:
        Kilonova = Kilonova[:data[Start:].shape[0]]
    else:
        temp = np.zeros(data[Start:].shape[0])
        temp[:Kilonova.shape[0]] = Kilonova
        Kilonova = temp
    testdata = np.copy(data)
    testdata[Start:,Position[0],Position[1]] = testdata[Start:,Position[0],Position[1]] + Kilonova
    
    Detections, Fakes, Masked = Kep_test(VARIABLES)
    out = [Detections,Fakes,Masked]
    writemyprog(filename,out)
    mytimestop = time.time()
    mytime = mytimestop-mytimestart
    print 'n=%g' %n, 'my_time=%f' %mytime


# MPI collective communication (all reduce)

stop = time.time()
print_master('Time taken=%f' %(stop-start))

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-9-d64cbd5bbfcc>, line 18)